In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki, POS tagged

sf = open(datapath+'/Corpora/wiki/simple_20200601/Tagged/simple_20200601_tagged.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April|NNP is|VBZ the|DT fourth|JJ month|NN of|IN the|DT year|NN and|CC comes|VBZ between|IN March|NNP and|CC May|NNP 

It|PRP is|VBZ one|CD of|IN four|CD months|NNS to|TO have|VB 30|CD days|NNS 

April|NNP always|RB begins|VBZ on|IN the|DT same|JJ day|NN of|IN week|NN as|IN July|NNP and|CC additionally|RB January|NNP in|IN leap|NNP years|NNS 

April|NNP always|RB ends|VBZ on|IN the|DT same|JJ day|NN of|IN the|DT week|NN as|IN December|NNP 

April|NNP 's|POS flowers|NNS are|VBP the|DT Sweet|NNP Pea|NN and|CC Daisy|NNP 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

from nltk.corpus.reader.util import read_line_block

In [5]:
simp_t = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/Tagged','simple_20200601_tagged.txt',
                            word_tokenizer = WhitespaceTokenizer(),
                            para_block_reader=read_line_block
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [7]:
from nltk.util import ngrams, everygrams

In [8]:

i = 0
for sent in word_gen(simp_t, remchars = "[`¬@^\"]"):
    i += 1
    print(sent)
    if i > 20 :break

April|NNP
is|VBZ
the|DT
fourth|JJ
month|NN
of|IN
the|DT
year|NN
and|CC
comes|VBZ
between|IN
March|NNP
and|CC
May|NNP
|^|
It|PRP
is|VBZ
one|CD
of|IN
four|CD
months|NNS


In [9]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [10]:
scorer = NgramAssocMeasures.poisson_stirling

In [25]:
tri_cf = TrigramCollocationFinder.from_words(word_gen(simp_t,remchars = "[`¬@^\"]"))
tri_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [26]:
bi_cf = tri_cf.bigram_finder()
bi_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [27]:
print("\t", [" ".join(tup) for tup in bi_cf.nbest(scorer, 15)])

	 ['0,155,117|CD 208,28,31|CD', '0,28|CD 1,76|CD', '0,688|CD 13603|CD', '0,724|CD 13604|CD', '0.000000535645|CD 5.35645|CD', '0.001696|CD 53.53|CD', '0.003733|CD 115.79|CD', '0.042|CD 0.095|CD', '0.11494204485329620070104015746959874|CD Kepler|NN', '0.18785964246206712024851793405427323|CD MRB|NN', '0.2115|CD 1975/2000|CD', '0.26149721284764278375542683860869585|CD Meissel|NNP', '0.28878809508660242127889972192923078|CD Flajolet|NNP', '0.2943958|CD 0.0000002|CD', '0.3075|CD 0.4667|CD']


In [28]:
bi_dict = {}
tri_dict = {}

In [29]:
for bigram in bi_cf.score_ngrams(scorer):    
    bi_dict[bigram[0]] = [bigram[0], bi_cf.ngram_fd[bigram[0]], bigram[-1]]
    
bi_dict

{('0,155,117|CD', '208,28,31|CD'): [('0,155,117|CD', '208,28,31|CD'),
  1,
  -26.011518439325844],
 ('0,28|CD', '1,76|CD'): [('0,28|CD', '1,76|CD'), 1, -26.011518439325844],
 ('0,688|CD', '13603|CD'): [('0,688|CD', '13603|CD'), 1, -26.011518439325844],
 ('0,724|CD', '13604|CD'): [('0,724|CD', '13604|CD'), 1, -26.011518439325844],
 ('0.000000535645|CD', '5.35645|CD'): [('0.000000535645|CD', '5.35645|CD'),
  1,
  -26.011518439325844],
 ('0.001696|CD', '53.53|CD'): [('0.001696|CD', '53.53|CD'),
  1,
  -26.011518439325844],
 ('0.003733|CD', '115.79|CD'): [('0.003733|CD', '115.79|CD'),
  1,
  -26.011518439325844],
 ('0.042|CD', '0.095|CD'): [('0.042|CD', '0.095|CD'), 1, -26.011518439325844],
 ('0.11494204485329620070104015746959874|CD',
  'Kepler|NN'): [('0.11494204485329620070104015746959874|CD',
   'Kepler|NN'), 1, -26.011518439325844],
 ('0.18785964246206712024851793405427323|CD',
  'MRB|NN'): [('0.18785964246206712024851793405427323|CD',
   'MRB|NN'), 1, -26.011518439325844],
 ('0.2115|

In [30]:
for trigram in tri_cf.score_ngrams(scorer):    
    tri_dict[trigram[0]] = [trigram[0], tri_cf.ngram_fd[trigram[0]], trigram[-1]]
    
tri_dict

{('0.001696|CD',
  '53.53|CD',
  '85.03|CD'): [('0.001696|CD',
   '53.53|CD',
   '85.03|CD'), 1, -26.011518439325844],
 ('0.003733|CD',
  '115.79|CD',
  '119.93|CD'): [('0.003733|CD',
   '115.79|CD',
   '119.93|CD'), 1, -26.011518439325844],
 ('0.401|CD', '0.823|CD', '1.385|CD'): [('0.401|CD', '0.823|CD', '1.385|CD'),
  1,
  -26.011518439325844],
 ('0.571|CD', '1.388|CD', '0.041|CD'): [('0.571|CD', '1.388|CD', '0.041|CD'),
  1,
  -26.011518439325844],
 ('002Tyrrell|NN',
  '003Tyrrell|NN',
  '004Tyrrell|NN'): [('002Tyrrell|NN',
   '003Tyrrell|NN',
   '004Tyrrell|NN'), 1, -26.011518439325844],
 ('005625|CD',
  '00650,555|CD',
  '650,555|CD'): [('005625|CD',
   '00650,555|CD',
   '650,555|CD'), 1, -26.011518439325844],
 ('01-07-2003|CD',
  '01-07-2004|CD',
  '01-07-2005|CD'): [('01-07-2003|CD',
   '01-07-2004|CD',
   '01-07-2005|CD'), 1, -26.011518439325844],
 ('01-07-2004|CD',
  '01-07-2005|CD',
  '01-07-2006|CD'): [('01-07-2004|CD',
   '01-07-2005|CD',
   '01-07-2006|CD'), 1, -26.011518

In [31]:
bigram_df = pd.DataFrame.from_dict(bi_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

trigram_df = pd.DataFrame.from_dict(tri_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

bigram_df

,ngram,freq,poisson
"(0,155,117|CD, 208,28,31|CD)","(0,155,117|CD, 208,28,31|CD)",1,-2.601152e+01
"(0,28|CD, 1,76|CD)","(0,28|CD, 1,76|CD)",1,-2.601152e+01
"(0,688|CD, 13603|CD)","(0,688|CD, 13603|CD)",1,-2.601152e+01
"(0,724|CD, 13604|CD)","(0,724|CD, 13604|CD)",1,-2.601152e+01
"(0.000000535645|CD, 5.35645|CD)","(0.000000535645|CD, 5.35645|CD)",1,-2.601152e+01
...,...,...,...
"(and|CC, the|DT)","(and|CC, the|DT)",50305,-2.524400e+06
"(to|IN, the|DT)","(to|IN, the|DT)",62529,-3.014295e+06
"(is|VBZ, a|DT)","(is|VBZ, a|DT)",83637,-3.950480e+06
"(in|IN, the|DT)","(in|IN, the|DT)",157334,-7.619869e+06


In [32]:
ngram_df = bigram_df.append(trigram_df).sort_values('poisson', ascending=False).reset_index(drop=True)

In [33]:
ngram_df['len'] = ngram_df.ngram.apply(len)

In [34]:
ngram_df

,ngram,freq,poisson,len
0,"(0,155,117|CD, 208,28,31|CD)",1,-2.601152e+01,2
1,"(μι|FW, αν|FW)",1,-2.601152e+01,2
2,"(νdαjμε̱|NNP, Κε̱τȣ|NNP)",1,-2.601152e+01,2
3,"(μὴ|NN, εἰσενέγκῃς|NN)",1,-2.601152e+01,2
4,"(μὴ|FW, ἀπόληται|FW)",1,-2.601152e+01,2
...,...,...,...,...
23923026,"(and|CC, the|DT)",50305,-2.524400e+06,2
23923027,"(to|IN, the|DT)",62529,-3.014295e+06,2
23923028,"(is|VBZ, a|DT)",83637,-3.950480e+06,2
23923029,"(in|IN, the|DT)",157334,-7.619869e+06,2


In [35]:
# Pickle ngram_df - can re-use later with different approaches (ngram_df = pd.read_pickle() )
ngram_df.to_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_df.pkl')

In [11]:
# ngram_df = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_df.pkl')

In [36]:
len(simp_t.words())

31748432

In [51]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(simp_t, remchars = "[`¬@^\"]", sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [14]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 's|POS',
 'that|IN',
 'that|WDT',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN',
 '|HYPH',
 '–|SYM'}

In [12]:
import pickle

In [54]:
with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop.pkl', 'wb') as pfile:
    pickle.dump(stop,pfile)
    

In [13]:
with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [15]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [16]:
min_freq = 20

In [17]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

ngram_df2

,ngram,freq,poisson,len
0,"(Grădina|NNP, Zoologică|NNP)",20,-6.066689e+02,2
1,"(Karlovy|NNP, Vary|NNP)",20,-6.066689e+02,2
2,"(Ving|NNP, Rhames|NNP)",20,-6.066689e+02,2
3,"(waystations|NNS, shuku|NN)",20,-6.080767e+02,2
4,"(Aer|NNP, Lingus|NNP)",20,-6.094190e+02,2
...,...,...,...,...
229281,"(and|CC, the|DT)",50305,-2.524400e+06,2
229282,"(to|IN, the|DT)",62529,-3.014295e+06,2
229283,"(is|VBZ, a|DT)",83637,-3.950480e+06,2
229284,"(in|IN, the|DT)",157334,-7.619869e+06,2


In [18]:
eval_count = 150000

ngram_eval = ngram_df2[0:eval_count]

ngram_eval

,ngram,freq,poisson,len
0,"(Grădina|NNP, Zoologică|NNP)",20,-606.668931,2
1,"(Karlovy|NNP, Vary|NNP)",20,-606.668931,2
2,"(Ving|NNP, Rhames|NNP)",20,-606.668931,2
3,"(waystations|NNS, shuku|NN)",20,-608.076717,2
4,"(Aer|NNP, Lingus|NNP)",20,-609.419001,2
...,...,...,...,...
149995,"(Cena|NNP, and|CC)",56,-2732.579707,2
149996,"(and|CC, keeping|VBG)",56,-2732.579707,2
149997,"(province|NN, has|VBZ, a|DT)",40,-2732.586816,3
149998,"(speak|VBP, a|DT)",56,-2732.604570,2


In [19]:
batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 15)

batches

{('Grădina|NNP', 'Zoologică|NNP'): 1,
 ('Karlovy|NNP', 'Vary|NNP'): 1,
 ('Ving|NNP', 'Rhames|NNP'): 1,
 ('waystations|NNS', 'shuku|NN'): 1,
 ('Aer|NNP', 'Lingus|NNP'): 1,
 ('Edinson|NNP', 'Cavani|NNP'): 1,
 ('Mirella|NNP', 'Freni|NNP'): 1,
 ('Flèche|NNP', 'Wallonne|NNP'): 1,
 ('Helicobacter|NNP', 'pylori|NNP'): 1,
 ('Sheela|NNP', 'Shrinivas|NNP'): 1,
 ('Mesut|NNP', 'Özil|NNP'): 1,
 ('Sonu|NNP', 'Nigam|NNP'): 1,
 ('Päijänne|NNP', 'Tavastia|NNP'): 1,
 ('Bellas|NNP', 'Artes|NNP'): 1,
 ('Bundesarchiv|NNP', 'Bild|NNP'): 1,
 ('Trisha|NNP', 'Yearwood|NNP'): 1,
 ('Dany|NNP', 'Heatley|NNP'): 1,
 ('Pim|NNP', 'Fortuyn|NNP'): 1,
 ('Sächsische|NNP', 'Schweiz|NNP'): 1,
 ('Jomo|NNP', 'Kenyatta|NNP'): 1,
 ('BNP|NNP', 'Paribas|NNP'): 1,
 ('Tayyip|NNP', 'Erdoğan|NNP'): 1,
 ('Liev|NNP', 'Schreiber|NNP'): 1,
 ('conventionally|RB', 'delimit|VBP'): 1,
 ('Dukla|NNP', 'Jihlava|NNP'): 1,
 ('Lieke|NNP', 'Martens|NNP'): 1,
 ('Dag|NNP', 'Hammarskjöld|NNP'): 1,
 ('mutually|RB', 'intelligible|JJ'): 1,
 ('Jule|NNP',

In [20]:
# Should be -2 as all stopwords
batches[('1|CD', 'to|IN', '1|CD')]

-2

In [21]:
len(batches)

150000

In [22]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(Grădina|NNP, Zoologică|NNP)",1
1,"(Karlovy|NNP, Vary|NNP)",1
2,"(Ving|NNP, Rhames|NNP)",1
3,"(waystations|NNS, shuku|NN)",1
4,"(Aer|NNP, Lingus|NNP)",1
...,...,...
149995,"(Cena|NNP, and|CC)",5
149996,"(and|CC, keeping|VBG)",1
149997,"(province|NN, has|VBZ, a|DT)",6
149998,"(speak|VBP, a|DT)",2


In [23]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Grădina|NNP, Zoologică|NNP)",20,-606.668931,2,1
1,"(Karlovy|NNP, Vary|NNP)",20,-606.668931,2,1
2,"(Ving|NNP, Rhames|NNP)",20,-606.668931,2,1
3,"(waystations|NNS, shuku|NN)",20,-608.076717,2,1
4,"(Aer|NNP, Lingus|NNP)",20,-609.419001,2,1
...,...,...,...,...,...
149995,"(Cena|NNP, and|CC)",56,-2732.579707,2,5
149996,"(and|CC, keeping|VBG)",56,-2732.579707,2,1
149997,"(province|NN, has|VBZ, a|DT)",40,-2732.586816,3,6
149998,"(speak|VBP, a|DT)",56,-2732.604570,2,2


In [24]:
ngram_eval.to_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval.pkl')

In [25]:
ngram_eval.batch.value_counts()

 1     47772
 2     33213
 3     25419
 4     15895
 5     10517
 6      6533
 7      3782
 8      2398
 9      1434
 10     1073
-2       732
 11      317
-1       282
 14      239
 12      227
 13       96
 15       71
Name: batch, dtype: int64